In [3]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

DROP DATABASE IF EXISTS lkvdb CASCADE;

CREATE DATABASE lkvdb LOCATION '/data/stackexchange1000';

USE lkvdb;

DROP TABLE IF EXISTS posts_sample;

CREATE EXTERNAL TABLE posts_sample(
    Id STRING,
    PostTypeId STRING,
    ParentId STRING,
    CreationDate STRING
)
ROW FORMAT
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
"input.regex" = '(?=.*\\bId=\"(\\d*)\")?(?=.*\\bParentId=\"(\\S*)\")?(?=.*\\bPostTypeId=\"(\\S*)\")?(?=.*\\bCreationDate=\"(\\S*)\")?.*$'
)
LOCATION '/data/stackexchange1000/posts'

tblproperties ("skip.header.line.count"="1");

DROP VIEW IF EXISTS posts_sample_view;

CREATE VIEW posts_sample_view(
    Id,
    PostTypeId,
    ParentId,
    YEAR,
    MONTH
)
PARTITIONED ON (YEAR, MONTH)
AS SELECT
    Id,
    PostTypeId,
    ParentId,
    regexp_extract(CreationDate, "[0-9]{4}", 0),
    regexp_extract(CreationDate, "[0-9]{4}-[0-9]{2}", 0)
FROM posts_sample;

SELECT YEAR, MONTH, count FROM (
    SELECT * FROM (
        SELECT 
            YEAR, 
            MONTH, 
            count(*) as count, 
            row_number() over() as rn 
        FROM posts_sample_view 
        GROUP BY YEAR, MONTH 
        ORDER BY rn DESC 
        LIMIT 3
    ) AS t 
    ORDER BY t.rn ASC 
    LIMIT 1
) AS tt;

Overwriting query.hql


In [4]:
!hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.059 seconds
OK
Time taken: 0.333 seconds
OK
Time taken: 0.027 seconds
OK
Time taken: 0.08 seconds
OK
Time taken: 0.624 seconds
OK
Time taken: 0.023 seconds
OK
Time taken: 1.143 seconds
Query ID = jovyan_20180303174949_d8498229-1128-4c6e-960b-ae664cbdc945
Total jobs = 4
Launching Job 1 out of 4
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1520090433404_0016, Tra